In [52]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: Qt5Agg


In [53]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "example_data.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [54]:
N_passes = 20

instances_df = instances_500km_df.loc[0:N_passes,:]

In [55]:
from problems.latency_problem import LatencyProblem
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 30.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df['isend'].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = Requirements()
requirements.min_throughput = 5e9
#requirements.max_throughput = 50e9
#requirements.max_latency = 5400

problem = LatencyProblem(instances_df, sys_param, requirements=requirements)

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 1000, 'n_offsprings': 50, 'n_max_gen': 1000}

settings = {}

# NSGA-III
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.factory import get_reference_directions, get_visualization

ref_dirs = get_reference_directions("das-dennis", 4, n_partitions=8)
algo_settings['pop_size'] = int(np.ceil(len(ref_dirs) / 100) * 100)
algo_settings['n_offsprings'] = int(np.ceil(len(ref_dirs) / 100) * 100 / 4)

print("# dirs: %d" % len(ref_dirs))

settings['NSGA-III'] = {
    'label': 'nsga3',
    'algorithm': NSGA3(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=algo_settings['n_max_gen'],
                                                      n_max_evals=None)
}

setting = settings['NSGA-III']

# dirs: 165


In [56]:
termination = setting['termination']
algorithm = setting['algorithm']

n_gen_inc = 100

In [57]:
from os import path

file_name = 'no_max_latency_res_%s_%d.pkl' % (setting['label'], algo_settings['pop_size'])
if path.exists(file_name):
    problem, setting, res = pickle.load(open(file_name, 'rb'))
    algorithm = res.algorithm
    algorithm.has_terminated = False
    n_gen = res.algorithm.n_gen
else:
    n_gen = 0

In [58]:
n_gen = n_gen + n_gen_inc
res = minimize(problem,
               algorithm,
               ('n_gen', n_gen),
               seed=1,
               copy_algorithm=False,
               #save_history=True,
               verbose=True
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

pickle.dump((problem, setting, res), open(file_name, 'wb'))

    1 |     200 |  0.00000E+00 |  1.832061868 |      18 |  3.774249378 |        ideal
    2 |     250 |  0.00000E+00 |  1.365984530 |      19 |  0.106626723 |        nadir
    3 |     300 |  0.00000E+00 |  1.058548886 |      20 |  0.018201372 |        ideal
    4 |     350 |  0.00000E+00 |  0.793172568 |      19 |  0.119357441 |        ideal
    5 |     400 |  0.00000E+00 |  0.455359349 |      19 |  0.031094205 |            f
    6 |     450 |  0.00000E+00 |  0.237522813 |      20 |  0.354089806 |        ideal
    7 |     500 |  0.00000E+00 |  0.103469156 |      21 |  0.008737354 |            f
    8 |     550 |  0.00000E+00 |  0.010008539 |      22 |  0.035979819 |            f
    9 |     600 |  0.00000E+00 |  0.00000E+00 |      24 |  0.026895398 |            f
   10 |     650 |  0.00000E+00 |  0.00000E+00 |      25 |  0.024630647 |            f
   11 |     700 |  0.00000E+00 |  0.00000E+00 |      25 |  0.015156963 |        ideal
   12 |     750 |  0.00000E+00 |  0.00000E+00 |      2

In [59]:
print(file_name)

no_max_latency_res_nsga3_200.pkl
